<a href="https://colab.research.google.com/github/wesley-wms/dio_transfer_learning_em_python/blob/main/transfer_learning_cats_vs_dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.keras import layers, models

### Carrega o dataset cats_vs_dogs usando tensorflow_datasets

In [2]:
#https://www.tensorflow.org/datasets/api_docs/python/tfds/load

(ds_train, ds_val), ds_info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:]'],  # Divide os dados em 80% treino e 20% validação
    with_info=True,  # Fornece informações adicionais sobre o dataset (nome, versão e características)
    as_supervised=True  # Retorna apenas (imagem, label)
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/23262 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/incomplete.K8SMNY_4.0.1/cats_vs_dogs-train.tfrecord*...:   0%…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.


O redimensionamento para 160x160 pixels garante compatibilidade com o MobileNetV2.
Normalizar os pixels para [0, 1] acelera o treinamento.

In [4]:
# Dimensão das imagens de entrada esperadas pelo modelo
IMG_SIZE = (160, 160)

# Função para pré-processar as imagens
def preprocess(image, label):
    # Redimensionar a imagem para 160x160 pixels
    image = tf.image.resize(image, IMG_SIZE)
    # Normalizar os valores dos pixels para o intervalo [0, 1]
    image = image / 255.0
    return image, label

ds_train = ds_train.map(preprocess).batch(32).shuffle(1000) # embaralha os dados no conjunto de treinamento para reduzir viés (bayes)
ds_val = ds_val.map(preprocess).batch(32) #agrupa os dados em lotes para otimizar o treinamento

Carregar o modelo pré-treinado MobileNetV2

In [5]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(160, 160, 3), # Define a entrada do modelo como imagens RGB de 160x160
    include_top=False,  # Remove a camada de saída usada para ImageNet
    weights='imagenet'  # Usa os pesos treinados no dataset ImageNet
)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Congelar as camadas do modelo base para evitar que seus pesos sejam atualizados

In [6]:
base_model.trainable = False

Construir o modelo final adicionando camadas personalizadas

In [7]:
model = models.Sequential([
    base_model,  # Modelo base pré-treinado
    layers.GlobalAveragePooling2D(),  # Reduz as dimensões espaciais em uma média global
    layers.Dense(1, activation='sigmoid')  # Camada de saída com ativação sigmoide para classificação binária
])

Compila o modelo

In [8]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(), # ajusta os pesos do modelo de forma eficiente
    loss='binary_crossentropy', # classificação binária
    metrics=['accuracy'] # define a precisão como métrica de desempenho
)

Treinar o modelo

In [ ]:
history = model.fit(
    ds_train,  # Dados de treinamento
    validation_data=ds_val,  # Dados de validação
    epochs=3  # Número de épocas
)

Epoch 1/3
582/582 ━━━━━━━━━━━━━━━━━━━━ 515s 816ms/step - accuracy: 0.9422 - loss: 0.1476 - val_accuracy: 0.9783 - val_loss: 0.0600
Epoch 2/3
582/582 ━━━━━━━━━━━━━━━━━━━━ 0s 539ms/step - accuracy: 0.9768 - loss: 0.0584

Avaliar o modelo

In [ ]:
loss, accuracy = model.evaluate(ds_val)
print(f"Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")